# Astropy

### What is it?

A set of Python modules for astrophysics. It contains many useful functions for dealing with datasets, astronomical coordinates, astrometry, units, etc. The astropy project tries to maintain fairly high coding standards, and astropy modules are maintained and regularly updated.

### How do I install it?

If you're on a \*nix system, then you have a couple of ways of installing astropy. In order of my preference:

1. Using `pip`, which is a package manager for Python:
   `pip install --user astropy` *or* `sudo pip install astropy`.
2. Using your system's package manager. In Ubuntu/Debian, this would be
   `sudo apt-get install python-astropy`.
3. Using Anaconda, which is another package manager for Python: `conda install -c anaconda astropy`.

There are more instructions [here](http://docs.astropy.org/en/stable/install.html).

### And now, for something completely different

In this notebook, we'll briefly touch on a couple of astropy's sub-packages. There are many we won't cover, however. Take a look at the [astropy documentation](http://astropy.readthedocs.io/en/stable/index.html) to get an overview of the sub-packages they have.

## Let's import some useful packages first

In [ ]:
from __future__ import print_function, division # Do this for Python 2/3 compatibility

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# Units (astropy.units)

A number of astropy modules use astropy.units to keep track of units. This can help avoid headaches and bugs, because the quantities being passed around to different functions always have units attached to them. You won't have to wonder whether the variable `dist` is in units of parsecs or kiloparsecs. It will contain that information.

In [ ]:
import astropy.units as u

Let's define some arrays of distances:

In [ ]:
d1 = np.random.random(5) * u.m    # u.meter also works
d2 = np.random.random(5) * u.pc   # u.parsec also works

print(d1)
print(d2)

Let's take the ratio of these different distances:

In [ ]:
print(d2/d1)

We can ask astropy.units to evaluate `pc / m`:

In [ ]:
print((d2/d1).decompose())

Let's try out some different units of angle:

In [ ]:
theta1 = 1.7 * u.deg
theta2 = 25. * u.arcsec

print((theta1/theta2).decompose())

We can do conversions between coordinates:

In [ ]:
print('{:.5g} = {:.5g} = {:.5g}'.format(theta1, theta1.to(u.rad), theta1.to(u.arcmin)))

Notice as well how astropy.units plays nicely with Python string formatting in the line above.

We can convert any quantity to [cgs](https://en.wikipedia.org/wiki/Centimetre%E2%80%93gram%E2%80%93second_system_of_units) or [SI](https://en.wikipedia.org/wiki/International_System_of_Units):

In [ ]:
print(d2.si)
print(d2.cgs)

There are several other useful features of astropy.units, which we don't have time to go into here. Take a look at the documentation [here](http://astropy.readthedocs.io/en/stable/units/index.html).

# Constants (astropy.constants)

This module defines some physical constants.

In [ ]:
import astropy.constants as const

Let's take a look at the gravitational constant:

In [ ]:
print(const.G)

Notice how there are several pieces of information included here. We can take a look at the uncertainty, which is given in the same units as the constant itself:

In [ ]:
print(const.G.uncertainty)

Each constant is a `Quantity` object from astropy.units, so we can do all the things we saw above. For example, let's calculate the Schwarzschild radius of the Sun:

In [ ]:
r_s = const.G * const.M_sun / const.c**2

print('r_s = {:.3g} = {:.3g}'.format(r_s.to(u.km), r_s.to(u.AU)))

# Coordinates (astropy.coordinates)

This module allows you to convert between different spherical coordinate systems. Like astropy.constants, it uses astropy.units heavily.

In [ ]:
from astropy.coordinates import SkyCoord

There are a couple of different ways to define coordinates. We'll try out a few below:

In [ ]:
c1 = SkyCoord(100., 45., frame='galactic', unit='deg')
c2 = SkyCoord(60.*u.deg, -25.*u.deg, frame='galactic')
c3 = SkyCoord('10h50m35.3s', '+12d17m1.0s', frame='icrs')
c3 = SkyCoord('10:50:35.3', '+12:17:1.0', frame='icrs', unit=(u.hourangle, u.deg))

Notice how we've been defining a `frame` for each `SkyCoord` object. The `frame` is the celestial coordinate system. Some common systems are `galactic`, `icrs` and `fk5`. The latter two are different variants of equatorial coordinates.

We can also define arrays of coordinates:

In [ ]:
ra = 360. * np.random.random(5)
dec = 180. * np.random.random(5) - 90.

c4 = SkyCoord(ra*u.deg, dec*u.deg, frame='fk5')

print(c4)

We can transform betwee different coordinate systems:

In [ ]:
c4_gal = c4.transform_to('galactic')
c4_icrs = c4.transform_to('icrs')

print(c4_gal)
print(c4_icrs)

Let's extract the RA and Dec of `c4` in the ICRS system:

In [ ]:
print(c4_icrs.ra)
print(c4_icrs.ra.value)
print(c4_icrs.ra.unit)
print('')
print(c4_icrs.dec)
print(c4_icrs.dec.value)
print(c4_icrs.dec.unit)

Finally, we can include distances in sky coordinates, like so:

In [ ]:
c5 = SkyCoord(45.*u.deg, 10.*u.deg, distance=1.*u.kpc, frame='galactic')
print(c5)
print('d = {}'.format(c5.distance))
print('x, y, z = {}'.format(c5.cartesian))

One more cool thing in `astropy.coordinates`:

In [ ]:
from astropy.coordinates import EarthLocation
EarthLocation.of_site('Cerro Tololo Interamerican Observatory')

# ASCII Tables (astropy.io.ascii)

A veritable Swiss-army knife for ASCII tables.

In [ ]:
from astropy.io import ascii

The two basic functions are `ascii.read()` and `ascii.write()`.

People often distribute ASCII tables in their own pet format, so you end up writing custom code to read each new ASCII table you encounter. The function `ascii.read()` will attempt to figure out the format of the file by itself. This usually works, as we show below with three different files:

In [ ]:
data = ascii.read('ascii1.txt')
print(data)

In [ ]:
data = ascii.read('ascii2.txt')
print(data)

We can extract values from the table like so:

In [ ]:
print(data['obsid'][1])

With some files, `ascii.read()` is not able to guess the format, and needs some help:

In [ ]:
data = ascii.read('ascii3.txt')

We give `ascii.read()` some hints, and then it works:

In [ ]:
data = ascii.read('ascii3.txt', format='fixed_width', data_start=2)
print(data)

# FITS Files (astropy.io.fits)

[FITS](https://en.wikipedia.org/wiki/FITS) is a widely used data format for astronomical images and tables, and `astropy.io.fits` (which you might also know under its old name, `pyfits`) is a widely used Python library for manipulating FITS files.

In [ ]:
import astropy.io.fits as fits

Let's try to open a FITS file:

In [ ]:
fname = 'dss17460a27m.fits'
hdulist = fits.open(fname)

Let's see what's inside this FITS file:

In [ ]:
hdulist.info()

FITS files are organized into "Header-Data Units" ("HDUs" for short). Each HDU contains a header describing the structure, meaning, provenance, etc. of the data, followed by the data itself. There are different types of HDUs, among them: PrimaryHDU (the first HDU is always one of these), ImageHDU (containing an image), CompImageHDU (containing a compressed image), TableHDU (containing a table of data), BinTableHDU (ditto, but stored in a binary format).

The file we have has only one HDU, which is a PrimaryHDU.

We access a particular HDU just like we would get an item from a list. Let's look at the data in the first (and only) HDU in our FITS file:

In [ ]:
hdulist[0].data

Let's plot it using `matplotlib`:

In [ ]:
plt.imshow(hdulist[0].data, cmap='binary', interpolation='nearest');

Now, let's close this FITS file:

In [ ]:
hdulist.close()

Next, we'll be making our own file with table data. We begin by making a structured numpy array, which we'll fill with some arbitrary data:

In [ ]:
dtype = [
    ('x', 'f8'),
    ('y', 'f8'),
    ('velocity', 'f8'),
    ('objid', 'i4'),
    ('objname', 'S10')  # A string with a maximum length of 10 characters
]

data = np.empty(5, dtype=dtype)
data['x'][:] = np.random.random(5)
data['y'][:] = np.random.random(5)
data['velocity'][:] = np.random.random(5)
data['objid'][:] = np.arange(5)
data['objname'][:] = 'a b c d e'.split()

Now, we'll create an HDU that stores this data:

In [ ]:
hdu = fits.BinTableHDU(data=data)

Now, we'll create an HDUList (which could, in principle, have several HDUs, rather than just one), and we'll write it out to a file. The HDU requires a primary HDU, which we'll add in:

In [ ]:
hdulist = fits.HDUList([fits.PrimaryHDU(), hdu]) # We need a dummy primary HDU
hdulist.writeto('my_table.fits', clobber=True)  # clobber=True means that any existing file will be overwritten
hdulist.close()

That was just a very short intro to FITS files and `astropy.io.fits`. You'll probably have to heal with a lot of these files, and [the documentation](http://docs.astropy.org/en/stable/io/fits/index.html) will be very helpful.

# World Coordinate System (astropy.wcs)

The World Coordinate System (WCS) is a system for translating between coordinates in an astronomical image and coordinates on the sky. If you ever deal with raw images, you'll need to use WCS.

In [ ]:
import astropy.wcs as wcs

Normally, FITS images contain WCS information. We'll grab the WCS from the FITS image we opened earlier:

In [ ]:
fname = 'dss17460a27m.fits'
hdulist = fits.open(fname)

w = wcs.WCS(hdulist[0].header)
hdulist.close()

The two basic things WCS does is allow you to go from "world" (i.e., sky) coordinates to "pixel" (i.e., image) coordinates, and vice versa. First, we'll go from pixel to world coordinates:

In [ ]:
pix_coords = np.array([
    [10., 5.],
    [3., 8.],
    [20.5, 10.1]
])

world_coords = w.wcs_pix2world(pix_coords, 1)

print(world_coords)

In this case, the "world coordinates" are RA and Dec.

We can go from world coordinates to pixel coordinates as well:

In [ ]:
world_coords = np.array([
    [92.356, 20.4775],
    [92.353, 20.4779]
])

pix_coords = w.wcs_world2pix(world_coords, 1)

print(pix_coords)

# Convolution (astropy.convolve)

Astropy contains some useful convolution functions, allowing you to set kernels flexibly, and dealing with NaNs.

In [ ]:
import astropy.convolution as conv

Let's begin with our FITS image:

In [ ]:
hdulist = fits.open(fname)
img_orig = hdulist[0].data[:]
hdulist.close()

We define a kernel (there are many types we could choose), and then apply the convolution:

In [ ]:
kern = conv.Gaussian2DKernel(5.)

img_smooth = conv.convolve(img_orig, kern, boundary='extend')

Let's plot the results. The original is on the left, and the smoothed image on the right:

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(1,2,1)
ax.imshow(img_orig, cmap='binary', interpolation='nearest')
ax.axis('off')

ax = fig.add_subplot(1,2,2)
ax.imshow(img_smooth, cmap='binary', interpolation='nearest')
ax.axis('off');

One nice thing about `astropy.convolution` is that it deals with NaNs gracefully:

In [ ]:
img_orig[5:10, 5:10] = np.nan
img_smooth = conv.convolve(img_orig, kern, boundary='extend')

fig = plt.figure()

ax = fig.add_subplot(1,2,1)
ax.imshow(img_orig, cmap='binary', interpolation='nearest')
ax.axis('off')

ax = fig.add_subplot(1,2,2)
ax.imshow(img_smooth, cmap='binary', interpolation='nearest')
ax.axis('off')

## Astropy.cosmology
Cosmological quantities for standard and custom cosmologies.

In [ ]:
from astropy.cosmology import Planck15 as cosmo

In [ ]:
cosmo.comoving_distance(.2)

In [ ]:
cosmo.age(0)

In [ ]:
cosmo.kpc_comoving_per_arcmin(.5)

In [ ]:
redshifts = np.logspace(-3,3.2)
plt.semilogy(redshifts,cosmo.age(redshifts))
plt.xlabel('Redshift')
plt.ylabel('Age of Universe (Gyr)')
plt.vlines([0,1100],1e-4,1e1,linestyles='dashed')
plt.text(0,1e-3,'Now')
plt.text(1100,1e-3,'CMB')

Can also specify your own cosmology:

In [ ]:
from astropy.cosmology import FlatLambdaCDM
my_cosmo = FlatLambdaCDM(70, .3) #H0, omega_m 

In [ ]:
my_cosmo.age(0)